In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import HDF5Matrix 
import h5py
%matplotlib inline

In [ ]:
#f= h5py.File('Train/food_c101_n1000_r384x384x3.h5','r')

In [ ]:
xtrain=HDF5Matrix('Train/food_c101_n10099_r32x32x1.h5','images')
ytrain=HDF5Matrix('Train/food_c101_n10099_r32x32x1.h5','category')

In [ ]:
print ('xtrain shape is: ',xtrain.shape,', ytrain shape is: ',ytrain.shape)

In [ ]:
xtrain=np.asarray(xtrain,dtype='float32')

In [ ]:
ytrain=np.asarray(ytrain,dtype='float32')

In [ ]:
xtest=HDF5Matrix('Test/food_test_c101_n1000_r32x32x1.h5','images')
ytest=HDF5Matrix('Test/food_test_c101_n1000_r32x32x1.h5','category')

In [ ]:
print ('xtest shape is: ',xtest.shape,', ytest shape is: ',ytest.shape)

In [ ]:
xtest=np.asarray(xtest,dtype='float32')
ytest=np.asarray(ytest,dtype='float32')

In [ ]:
category_type=HDF5Matrix('Test/food_test_c101_n1000_r32x32x1.h5','category_names')

scale the data  

In [ ]:
xtrain_scaled=xtrain/255
xtest_scaled=xtest/255

In [ ]:
image_size=xtrain.shape[0]

# Network params

In [ ]:
n_channel=1
n_width=32
n_heght=32
n_output=101
BATCH_SIZE=64

In [ ]:
def create_variable(channel,width,height,output):
    X=tf.placeholder(tf.float32,shape=[None,height,width,channel],name='Input')
    Y=tf.placeholder(tf.float32,shape=[None,output],name='output')
    return X,Y

In [ ]:
def initialize_variable():
    W1=tf.get_variable("W1",[4,4,1,32],tf.float32,tf.contrib.layers.xavier_initializer(seed=0))
    W2=tf.get_variable("W2",[2,2,32,32],tf.float32,tf.contrib.layers.xavier_initializer(seed=0))
    W3=tf.get_variable("W3",[4,4,32,64],tf.float32,tf.contrib.layers.xavier_initializer(seed=0))
    W4=tf.get_variable("W4",[2,2,64,64],tf.float32,tf.contrib.layers.xavier_initializer(seed=0))
    return {'W1':W1,'W2':W2,'W3':W3,'W4':W4}
    

## Images Transformation

In [ ]:
def image_transformation(train):
    image=tf.map_fn(lambda img:tf.image.random_flip_left_right(img),train)
    image1=tf.map_fn(lambda img:tf.image.random_contrast(image,lower=0.2,upper=0.8),image)
    image2=tf.map_fn(lambda img:tf.image.random_brightness(image,max_delta=63),image1)
    #image3=tf.map_fn(lambda img:tf.image.per_image_standardization(image),image2)
    #image.set_shape([32,32,1])
    
    return image2

CNN Layers

In [ ]:
def create_layer(X,W_filter):
   
    
    #X,Y=create_variable(channel,width,height,output)
    mean_0,variance_0=tf.nn.moments(X,axes=[0,1,2],keep_dims=True)
    norm_X_1=tf.nn.batch_normalization(X,mean_0,variance_0,offset=0,scale=0,variance_epsilon=1e-6)
    
    conv1=tf.nn.conv2d(norm_X_1,W_filter['W1'],strides=[1,1,1,1],padding='SAME')
    max_pool1=tf.nn.max_pool(tf.nn.relu(conv1),ksize=[1,2,2,1],strides=[1,1,1,1],padding='SAME')
    dropout_1=tf.contrib.layers.dropout(max_pool1,keep_prob=0.2,is_training=True)
    
    norm1=tf.nn.lrn(dropout_1,depth_radius=4,bias=1.0,alpha=0.001/9.0,beta=0.4,name='norm1')
    
    
    
    conv2=tf.nn.conv2d(norm1,W_filter['W2'],strides=[1,2,2,1],padding='SAME')
    max_pool2=tf.nn.max_pool(tf.nn.relu(conv2),ksize=[1,4,4,1],strides=[1,2,2,1],padding='SAME')
    dropout_2=tf.contrib.layers.dropout(max_pool2,keep_prob=0.4,is_training=True)
    
    conv3=tf.nn.conv2d(dropout_2,W_filter['W3'],strides=[1,1,1,1],padding='SAME')
    max_pool3=tf.nn.max_pool(tf.nn.relu(conv3),ksize=[1,2,2,1],strides=[1,1,1,1],padding='SAME')
    dropout_3=tf.contrib.layers.dropout(max_pool3,keep_prob=0.2,is_training=True)
    
    
    
    mean_3,variance_3=tf.nn.moments(dropout_3,axes=[0,1,2],keep_dims=True)
    dropout_3_norm=tf.nn.batch_normalization(dropout_3,mean_3,variance_3,offset=0,scale=0,variance_epsilon=1e-6)
    
    conv4=tf.nn.conv2d(dropout_3_norm,W_filter['W4'],strides=[1,2,2,1],padding='SAME')
    max_pool3=tf.nn.max_pool(tf.nn.relu(conv4),ksize=[1,4,4,1],strides=[1,2,2,1],padding='SAME')
    dropout_4=tf.contrib.layers.dropout(max_pool3,keep_prob=0.4,is_training=True)
    
    norm2=tf.nn.lrn(dropout_4,depth_radius=4,bias=1.0,alpha=0.001/9.0,beta=0.4,name='norm2')
    
    flatten=tf.contrib.layers.flatten(norm2)
    out1=tf.contrib.layers.fully_connected(flatten,5000)
    dropout_out1=tf.contrib.layers.dropout(out1,keep_prob=0.2,is_training=True)
    
    out=tf.contrib.layers.fully_connected(out1,n_output,activation_fn=None)
    #out=tf.nn.softmax(out1)
    
    return out

In [ ]:
def compute_cost(out,Y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out,labels=Y))
    return cost

In [ ]:
def model():
    learning_rate=0.0001
    tf.reset_default_graph()
    #tf.get_default_graph()
    
    x,y=create_variable(n_channel,n_width,n_heght,n_output)
    W_dict=initialize_variable()
    out = create_layer(x,W_dict)
    loss=compute_cost(out,y)
    global_step=tf.Variable(0)
    learning_rate_1=tf.train.exponential_decay(0.5,global_step,5000,0.96,staircase=True)
    optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate_1).minimize(loss,global_step=global_step)
    #accuracy
    
    correct_pred = tf.equal(tf.argmax(out,axis=1),tf.argmax(y,axis=1))
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))
    init_op=tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
    
    with tf.Session() as sess:
        sess.run(init_op)
        
        for _ in range(20):
        
            for i in range(0,image_size,BATCH_SIZE):
                pos = (i % int(image_size / BATCH_SIZE)) * BATCH_SIZE
                #indices=np.random.choice(xtrain_scaled.shape[0],128)
                xtrain_batch,ytrain_batch=xtrain[pos:pos+BATCH_SIZE],ytrain[pos:pos+BATCH_SIZE]
                #xtrain_batch=image_transformation(xtrain_batch)
           
                opt,cost=sess.run([optimizer,loss],feed_dict={x:xtrain_batch,y:ytrain_batch})
        predictions=out.eval(feed_dict={x:xtest})    
        print ("Test Accuracy ",accuracy.eval(feed_dict={x:xtest,y:ytest}))
        print ("Train Accuracy ",accuracy.eval(feed_dict={x:xtrain,y:ytrain}))
        return predictions

In [ ]:
pred=model()

In [ ]:
np.argmax(pred,axis=1).shape

In [ ]:
np.argmax(ytest,axis=1)


In [ ]:
correct=[item for item in np.equal(np.argmax(pred,axis=1),np.argmax(ytest,axis=1)) if item == True ]

In [ ]:
correct

In [ ]:
ytest[94]

In [ ]:
category_type[94]

In [ ]:
ytrain[94]